### Imports

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

import os.path
import pickle
import tqdm

### Glicko Functions

In [13]:
def expected_placement_glicko(RATING, ALL_RATING, RD, ALL_RD, q):
  expected_placement = len(ALL_RATING) + 1

  for i in range(len(ALL_RATING)):
    expected_placement -= 1 / (1 + 10 ** ((-g(math.sqrt(RD**2 + ALL_RD[i]**2), q)*(RATING - ALL_RATING[i]))/400))

  return expected_placement - 0.5

def update_rating_rd_glicko(RATING, ALL_RATING, RD, ALL_RD, q, PLACEMENT):
    phi_sum = 0
    theta_sum = 0

    found_self = False
    for i in range(len(ALL_RATING)):
        if i + 1 != PLACEMENT:
            E = 1/(1 + 10**(-(g(math.sqrt(RD**2  + ALL_RD[i]**2), q)) * (RATING - ALL_RATING[i])/400))
            phi_sum += g(ALL_RD[i], q)*g(ALL_RD[i], q)*E*(1-E)
            theta_sum += g(ALL_RD[i], q)*((PLACEMENT <= i + 1) - E)
    d2 = 1/(q*q*phi_sum)
    return (RATING + theta_sum*q/(1/RD**2+1/d2), max(30,math.sqrt(1/(1/(RD**2)+1/d2))))

def g(RD, q):
    return 1/(math.sqrt(1 + (3*q*q*RD*RD)/(math.pi**2)))

# Import Event History

In [14]:
event_history = pd.read_json('../data/event_history.json')

# Calculate Rating

In [15]:
event_types = ["MCC", "BW", "FW", "PB", "MH", "MN", "BC", "FUW", "JC", "CT", "BB", "CH", "TT", "KG", "SD", "CC", "CCC"]

q = math.log(10)/400
c = 15
d = 20
starting_rating = 1000.0
starting_rd = 200.0

rating_history = pd.DataFrame(columns = np.concatenate((np.array(["Avatar", "Player", "Rating", "Peak", "Events", "I", "S"]), event_history['event'].values)))
rd_history = pd.DataFrame(columns = np.concatenate((np.array(["Player", "RD", "S"]), event_history['event'].values)))

summaries_unclassified = dict()

summaries = dict()
for event_type in event_types:
    summaries[event_type] = dict()

# Run this loop for every event

for i in tqdm.tqdm(range(len(event_history))):
    event_name = event_history.at[i, "event"]
    this_event = dict(event_history.iloc[i])
    this_event_results = pd.DataFrame(columns = np.array(["Position", "Player", "Rating", "Global", "EP", "AP", "Score", "Rating Change", "New Rating", "New Global", "Global Change"]))

    playing = []
    prior_rating = {}
    prior_rd = {}
    expected = {}
    actual = {}
    new_rating = {}
    new_rd = {}

    # Register current players

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")
        playing.append(player_id)

    for j in range(len(rd_history)):
        player_id = rd_history.iat[j, 0].lower().replace("_", "")

        # Update Rating Deviation

        this_rd = max(30,min(200, math.sqrt(rd_history.loc[player_id]["RD"]**2 + c**2)))
        rd_history.at[player_id, event_name] = this_rd
        rd_history.at[player_id, "RD"] = this_rd
        
        #  Rating Decay

        if player_id not in playing:
            this_rating = rating_history.at[player_id, "Rating"]
            this_inactivity = rating_history.at[player_id, "I"]
            rating_history.loc[player_id, event_name] = this_rating - (this_inactivity/5000)*(this_rating - starting_rating)
            rating_history.loc[player_id, "Rating"] = this_rating - (this_inactivity/5000)*(this_rating - starting_rating)
            rating_history.loc[player_id, "I"] += 1

    j = 1
    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        # Register New Players

        if player_id not in rating_history.index:
            rating_history.loc[player_id] = {
                "Avatar" : "https://mc-heads.net/avatar/" + player,
                "Player" : player,
                "I" : 0,
                "S" : starting_rating,
                "Rating" : starting_rating,
                "Events" : 0,
                "Peak": starting_rating
            }
            rd_history.loc[player_id] = {
                "Player" : player,
                "S" : starting_rd,
                "RD" : starting_rd
            }

        # Register prior stats

        prior_rating[player_id] = rating_history.at[player_id, "Rating"]
        prior_rd[player_id] = rd_history.at[player_id, "RD"]

        # Register placement

        actual[player_id] = j
        j += 1

    prior_rankings = rating_history.sort_values(by = "Rating", ascending = False)

    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        # Calculate Expected

        expected[player_id] = expected_placement_glicko(
            prior_rating[player_id], 
            list(prior_rating.values()), 
            prior_rd[player_id], 
            list(prior_rd.values()), 
            q
        )

        # Calculate and Update Rating and RD

        new_rating[player_id], new_rd[player_id] = update_rating_rd_glicko(
            prior_rating[player_id], 
            list(prior_rating.values()), 
            prior_rd[player_id], 
            list(prior_rd.values()), 
            q, 
            actual[player_id]
        )

        rating_history.at[player_id, "Rating"] = new_rating[player_id]
        rd_history.at[player_id, "RD"] = new_rd[player_id]
        rating_history.at[player_id, event_name] = new_rating[player_id]
        rd_history.at[player_id, event_name] = new_rd[player_id]
        rating_history.at[player_id, "Events"] += 1
        rating_history.at[player_id, "I"] = 0

        if rating_history.at[player_id, "Peak"] < new_rating[player_id]:
            rating_history.at[player_id, "Peak"] = new_rating[player_id]

    new_rankings = rating_history.sort_values(by = "Rating", ascending = False)

    j = 1
    for player in this_event['players']:
        player_id = player.lower().replace("_", "")

        this_event_results.loc[player_id] = {
            "Position" : j,
            "Player" : player,
            "Rating" : prior_rating[player_id],
            "Global" : prior_rankings.index.get_loc(player_id) + 1,
            "EP" : expected[player_id],
            "AP" : actual[player_id],
            "Score" : expected[player_id] - actual[player_id],
            "Rating Change" : new_rating[player_id] - prior_rating[player_id],
            "New Rating" : new_rating[player_id],
            "New Global" : new_rankings.index.get_loc(player_id) + 1,
            "Global Change" : prior_rankings.index.get_loc(player_id) - new_rankings.index.get_loc(player_id),
        }
        
        j += 1

    this_event['results'] = this_event_results

    if event_name[:2] in event_types:
        summaries[event_name[:2]][event_name] = this_event
    else:
        summaries[event_name[:3]][event_name] = this_event

    summaries_unclassified[event_name] = this_event

rating_history = rating_history.ffill(axis = 1)
rd_history = rd_history.ffill(axis = 1)
rankings = rating_history.sort_values(by = "Rating", ascending = False)[~rating_history["Player"].isin(["xShatter", "DONG_FORTNITE"])]

100%|██████████| 263/263 [03:07<00:00,  1.40it/s]
/tmp/ipykernel_7983/1368823515.py:158: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rankings = rating_history.sort_values(by = "Rating", ascending = False)[~rating_history["Player"].isin(["xShatter", "DONG_FORTNITE"])]


# Export Results

In [16]:
with open('../data/summaries.pkl', 'wb') as file:
    pickle.dump(summaries, file)

with open('../data/summaries_unclassified.pkl', 'wb') as file:
    pickle.dump(summaries_unclassified, file)

rating_history.to_parquet('../data/rating_history.parquet')
rd_history.to_parquet('../data/rd_history.parquet')
rankings.to_parquet('../data/rankings.parquet')